#### MLP Using Bayesian Optimisation on Boston Data

In [1]:
import numpy as np
import sklearn.gaussian_process as gp

from scipy.stats import norm
from scipy.optimize import minimize


def optimize(n_iters, loss_func, bounds, stop_func, x0=None, n_pre_samples=5,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7,):

    x_list = []
    y_list = []

    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(loss_func(params))
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(loss_func(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for _ in range(n_iters):

        model.fit(xp, yp)

        # Sample next hyperparameter
        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * expected_improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = sample_next_hyperparameter(expected_improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        # Sample loss for new set of parameters
        cv_score = loss_func(next_sample)

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)
        if stop_func(yp):
            print('Iteration stopped prematurely, reason: stop criteria met')
            return xp, yp

    return xp, yp


def sample_next_hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):

    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))

        if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x


def expected_improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):

    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)

    return -1 * expected_improvement


In [4]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import mean_squared_error 



def loss_func(params):
    
    val = cross_val_score(MLPRegressor(hidden_layer_sizes=(int(np.round(params[0]))), activation='relu', alpha=params[1], batch_size='auto', solver='lbfgs', max_iter=600),
                         X=X_train, y=y_train, scoring='neg_mean_squared_error', cv=5).mean()
                    
    print("Selected parameters are ", params)
    print("Value at selected parameters is, ", val)
    return val


def stop_func(y):
    if len(y) <= 10:
        return False
    y = y[-10:]
    max_val = np.max(y)
    min_val = np.min(y)
    # Can change the value here for other stop criteria
    if abs(max_val - min_val) < 1e-4:
        return True
    else:
        return False

boston = load_boston()
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
bounds = np.array([[1, 10], [0.0001, 0.001]])

xp, yp = optimize(n_iters=50,
                            loss_func=loss_func,
                            bounds=bounds,
                            stop_func=stop_func,
                            n_pre_samples=1,
                            random_search=100000)
indx = np.argmax(yp)
print('Best score found is ', yp[indx])
print('Best settings are found is ', xp[indx])

Selected parameters are  [8.55133310e+00 1.49231764e-04]
Value at selected parameters is,  -19.26565312196637
Selected parameters are  [1.00001674e+00 5.31975304e-04]
Value at selected parameters is,  -25.106899800611362
Selected parameters are  [8.55145623e+00 1.75039894e-04]
Value at selected parameters is,  -18.235125257519584
Selected parameters are  [8.55123650e+00 2.52383858e-04]
Value at selected parameters is,  -22.307842501789306
Selected parameters are  [8.55170575e+00 2.48133357e-04]
Value at selected parameters is,  -24.20231379903165
Selected parameters are  [9.36297888e+00 4.99723643e-04]
Value at selected parameters is,  -23.302270133507513
Selected parameters are  [8.55151466e+00 2.71168415e-04]
Value at selected parameters is,  -20.696173491992802
Selected parameters are  [8.55136162e+00 1.16220516e-04]
Value at selected parameters is,  -19.425200035307885
Selected parameters are  [8.55130110e+00 1.27322125e-04]
Value at selected parameters is,  -19.082663131163862
Sel

C:\Users\puru_\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.13856519e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\Users\puru_\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.05891063e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)


Selected parameters are  [8.55147694e+00 2.36050184e-04]
Value at selected parameters is,  -18.86164576020468
Selected parameters are  [9.16869042e+00 1.29747920e-04]
Value at selected parameters is,  -19.93550731826675
Selected parameters are  [9.16876453e+00 1.37497028e-04]
Value at selected parameters is,  -19.310542572200735
Selected parameters are  [8.55144234e+00 2.76057177e-04]
Value at selected parameters is,  -16.07306117736503
Selected parameters are  [8.55145148e+00 2.60069819e-04]
Value at selected parameters is,  -16.87632942022315
Selected parameters are  [9.16840390e+00 1.71863598e-04]
Value at selected parameters is,  -17.154245677610717
Selected parameters are  [8.55130608e+00 1.03125305e-04]
Value at selected parameters is,  -25.20720805317157
Selected parameters are  [9.16840659e+00 2.15359230e-04]
Value at selected parameters is,  -21.36169843958185
Selected parameters are  [9.16878604e+00 1.71309946e-04]
Value at selected parameters is,  -16.725080340524112


C:\Users\puru_\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([3.29298384e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


Selected parameters are  [9.16873493e+00 1.52758986e-04]
Value at selected parameters is,  -19.636482737133612
Selected parameters are  [9.16883393e+00 2.25845001e-04]
Value at selected parameters is,  -19.83480389020182
Selected parameters are  [9.16860977e+00 1.33580145e-04]
Value at selected parameters is,  -21.774598647887395
Selected parameters are  [5.99396542e+00 6.12833789e-04]
Value at selected parameters is,  -17.938086437230318
Selected parameters are  [9.16869390e+00 3.56928579e-04]
Value at selected parameters is,  -23.35762534432762
Selected parameters are  [5.99414784e+00 6.39350930e-04]
Value at selected parameters is,  -23.20460376112054
Selected parameters are  [9.16881024e+00 2.89852717e-04]
Value at selected parameters is,  -21.26304192562522
Selected parameters are  [8.55141919e+00 1.96342497e-04]
Value at selected parameters is,  -21.473523131813636
Selected parameters are  [8.55128678e+00 1.34245535e-04]
Value at selected parameters is,  -22.140851223735762
Selec

C:\Users\puru_\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([6.14225725e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


Selected parameters are  [8.55147835e+00 2.19439084e-04]
Value at selected parameters is,  -19.42554114308679
Selected parameters are  [9.16866627e+00 1.72650931e-04]
Value at selected parameters is,  -19.2489504239401
Selected parameters are  [5.99401038e+00 6.46334370e-04]
Value at selected parameters is,  -16.14856967478893
Selected parameters are  [5.99394059e+00 6.36585131e-04]
Value at selected parameters is,  -19.550524903163186
Selected parameters are  [7.67170124e+00 4.19680885e-04]
Value at selected parameters is,  -16.42926481615205
Selected parameters are  [9.16885817e+00 2.53842214e-04]
Value at selected parameters is,  -18.026398477026316
Selected parameters are  [5.99392257e+00 6.44792723e-04]
Value at selected parameters is,  -22.836992720053974
Selected parameters are  [9.16886094e+00 2.64102617e-04]
Value at selected parameters is,  -20.466870719122728
Selected parameters are  [9.16844955e+00 1.90140521e-04]
Value at selected parameters is,  -23.53902342415849
Selecte

In [5]:
mlp = MLPRegressor(hidden_layer_sizes=(int(np.round(xp[indx][0]))), activation='relu', alpha=xp[indx][1], batch_size='auto', solver='lbfgs', max_iter=600)
mlp.fit(X_train, y_train)
train_mse = mean_squared_error(y_train, mlp.predict(X_train))
test_mse = mean_squared_error(y_test, mlp.predict(X_test))
 
print("Train MSE:", np.round(train_mse, 2))
print("Test MSE:", np.round(test_mse, 2))

Train MSE: 3.59
Test MSE: 10.9


In [6]:
import plotly.plotly as py
import plotly.graph_objs as go
trace = go.Scatter(
    x = y_test,
    y = mlp.predict(X_test),
    mode = 'markers',
    name="Predicted vs Actual"
)
trace1 = go.Scatter(
    name="Best Fit",
    x = y_test,
    y = y_test,
    line = dict(
        color = ('rgb(0, 0, 0)'),
        width = 4,
        dash = 'dash')
)
data1 = [trace,trace1]

# Plot and embed in ipython notebook!
py.iplot(data1, filename='basic-scatter')